In [1]:
import torch
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset

class MRIDataset(Dataset):
    def __init__(self, path):
        self.x     = [] #MRI samples
        self.y1    = [] #Coordinate labels
        self.y2    = [] #Severity labels
        self.masks = [] #Label masks
        for study in os.listdir(path):
            study_path = os.path.join(path,study)
            # print(study_path)
            for series in os.listdir(study_path):
                data_path = f'{study_path}/{series}/{series}_'
                self.x.append(np.load(f'{data_path}image.npy'))
                self.y1.append(np.load(f'{data_path}coordinates.npy'))
                self.y2.append(np.load(f'{data_path}labels.npy'))
                self.masks.append(np.load(f'{data_path}mask.npy'))

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y1 = self.y1[idx]
        y2 = self.y2[idx]
        return {'x': x, 'y1': y1, 'y2': y2}

class MRIDataLoader(DataLoader):
    def __init__(self, hyperparameters):
        

path = '../../preprocessing/datasets/resampled-1.5mm_resolution/'

# Create dataset
dataset = MRIDataset(path)

# Create dataloader
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Iterate over the dataloader

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6259,) + inhomogeneous part.

In [7]:
x = [[234,23423],[234,23423],[234,23423]]
len(x)

3